In [4]:
import os
import nbformat
from nbformat.v4 import new_markdown_cell, new_code_cell

def load_notebook(path):
    with open(path, 'r', encoding='utf-8') as f:
        return nbformat.read(f, as_version=4)

def save_notebook(notebook, path):
    with open(path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)

def generate_cells_from_project(project_path):
    cells = []
    for root, _, files in os.walk(project_path):
        for file in sorted(files):
            if file.endswith(".py"):
                file_path = os.path.join(root, file)
                rel_path = os.path.relpath(file_path, project_path)
                with open(file_path, 'r', encoding='utf-8') as f:
                    code = f.read()
                cells.append(new_markdown_cell(f"### `{rel_path}`"))
                cells.append(new_code_cell(code))
    return cells

def inject_into_section(notebook, section_title, new_cells):
    cells = notebook.cells
    section_idx = None
    next_section_idx = len(cells)

    # Find the section header
    for i, cell in enumerate(cells):
        if cell.cell_type == "markdown":
            src = cell.source.strip()
            if src.startswith('#') and section_title in src:
                section_idx = i
                break

    if section_idx is not None:
        # Find where the section ends
        base_header_level = cells[section_idx].source.strip().count('#')
        for j in range(section_idx + 1, len(cells)):
            if cells[j].cell_type == "markdown":
                header_level = cells[j].source.strip().count('#')
                if header_level <= base_header_level:
                    next_section_idx = j
                    break
        # Replace the section
        notebook.cells = cells[:section_idx + 1] + new_cells + cells[next_section_idx:]
    else:
        # Append new section
        notebook.cells.append(new_markdown_cell(f"# {section_title}"))
        notebook.cells.extend(new_cells)

    return notebook

def update_notebook_with_project(project_path, notebook_path, section_title):
    notebook = load_notebook(notebook_path)
    project_cells = generate_cells_from_project(project_path)
    updated_notebook = inject_into_section(notebook, section_title, project_cells)
    save_notebook(updated_notebook, notebook_path)

# Example usage:
project_dir = "./src/"
notebook_path = "project_summary.ipynb"
section_header = "Functionalities"

update_notebook_with_project(project_dir, notebook_path, section_header)